In [3]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\sun_c\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [4]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [5]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [7]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [8]:
# Set features. This will also be used as your x values.
data = df.drop('koi_disposition', axis=1)
target = df['koi_disposition']
feature_names = data.columns
data.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    data, target, random_state=1)

In [10]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3563,0,0,0,0,10.548413,5.470000e-05,-5.470000e-05,139.064020,0.004110,-0.004110,...,-133,4.387,0.066,-0.123,1.092,0.181,-0.097,298.09543,44.737061,13.204
4099,0,0,0,0,24.754385,1.365000e-04,-1.365000e-04,140.207320,0.004460,-0.004460,...,-144,4.519,0.078,-0.052,0.804,0.056,-0.076,295.73535,42.576248,15.514
5460,0,0,0,0,1.057336,1.230000e-07,-1.230000e-07,131.792007,0.000096,-0.000096,...,-140,4.594,0.054,-0.027,0.683,0.054,-0.060,292.18417,49.310040,15.414
1091,0,0,0,0,201.118319,1.461000e-03,-1.461000e-03,187.569860,0.005290,-0.005290,...,-112,4.447,0.072,-0.108,0.954,0.135,-0.083,283.11377,48.131390,13.328
5999,0,0,0,0,91.649983,3.181000e-03,-3.181000e-03,175.715600,0.028600,-0.028600,...,-233,4.145,0.164,-0.164,1.608,0.905,-0.383,294.93198,39.812420,12.964


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [11]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model

In [16]:
# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)

In [19]:
print(rf.score(X_train_scaled, y_train))
print(rf.score(X_test_scaled, y_test))
print(rf.score(X_test, y_test))

1.0
0.9067505720823799
0.7488558352402745


In [18]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.10554803537030448, 'koi_fpflag_co'),
 (0.08998060552310225, 'koi_fpflag_nt'),
 (0.0667700880210715, 'koi_fpflag_ss'),
 (0.060102162561747885, 'koi_model_snr'),
 (0.051905654723694614, 'koi_prad'),
 (0.036264643359124576, 'koi_fpflag_ec'),
 (0.036035992006668065, 'koi_prad_err1'),
 (0.03314126012096086, 'koi_duration_err2'),
 (0.03282764268661779, 'koi_steff_err1'),
 (0.0324855582639982, 'koi_duration_err1'),
 (0.030828282353636376, 'koi_prad_err2'),
 (0.024809948879247767, 'koi_steff_err2'),
 (0.023653535772969393, 'koi_duration'),
 (0.023073802100991667, 'koi_time0bk_err2'),
 (0.022669820618309702, 'koi_period'),
 (0.021746100898163107, 'koi_time0bk_err1'),
 (0.020559161338830255, 'koi_depth'),
 (0.019519642318942817, 'koi_insol_err1'),
 (0.017877941350922612, 'koi_teq'),
 (0.01760950775282415, 'koi_period_err1'),
 (0.017170119769700676, 'koi_impact'),
 (0.016897516120103897, 'koi_period_err2'),
 (0.01553284678327369, 'koi_time0bk'),
 (0.014277716034900481, 'koi_depth_err2'),
 (0.

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [25]:
sorted(rf.get_params().keys())

['bootstrap',
 'ccp_alpha',
 'class_weight',
 'criterion',
 'max_depth',
 'max_features',
 'max_leaf_nodes',
 'max_samples',
 'min_impurity_decrease',
 'min_impurity_split',
 'min_samples_leaf',
 'min_samples_split',
 'min_weight_fraction_leaf',
 'n_estimators',
 'n_jobs',
 'oob_score',
 'random_state',
 'verbose',
 'warm_start']

In [28]:
# Create the GridSearchCV model
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': [20, 60, 80, None],
              'n_estimators': [300, 900, 1800, 2100]}
grid = GridSearchCV(rf, param_grid, verbose=3)

In [29]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ....max_depth=20, n_estimators=300;, score=0.881 total time=   3.3s
[CV 2/5] END ....max_depth=20, n_estimators=300;, score=0.897 total time=   3.4s
[CV 3/5] END ....max_depth=20, n_estimators=300;, score=0.896 total time=   3.4s
[CV 4/5] END ....max_depth=20, n_estimators=300;, score=0.893 total time=   3.4s
[CV 5/5] END ....max_depth=20, n_estimators=300;, score=0.875 total time=   3.4s
[CV 1/5] END ....max_depth=20, n_estimators=900;, score=0.884 total time=  10.5s
[CV 2/5] END ....max_depth=20, n_estimators=900;, score=0.900 total time=  10.7s
[CV 3/5] END ....max_depth=20, n_estimators=900;, score=0.899 total time=  10.6s
[CV 4/5] END ....max_depth=20, n_estimators=900;, score=0.891 total time=  10.6s
[CV 5/5] END ....max_depth=20, n_estimators=900;, score=0.877 total time=  10.7s
[CV 1/5] END ...max_depth=20, n_estimators=1800;, score=0.888 total time=  21.7s
[CV 2/5] END ...max_depth=20, n_estimators=1800;

GridSearchCV(estimator=RandomForestClassifier(n_estimators=200),
             param_grid={'max_depth': [20, 60, 80, None],
                         'n_estimators': [300, 900, 1800, 2100]},
             verbose=3)

In [30]:
# List the best parameters for this dataset
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 20, 'n_estimators': 1800}
0.8935703941958536


In [31]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.85      0.77      0.81       404
     CONFIRMED       0.82      0.87      0.84       435
FALSE POSITIVE       0.99      1.00      0.99       909

      accuracy                           0.91      1748
     macro avg       0.89      0.88      0.88      1748
  weighted avg       0.91      0.91      0.91      1748



# Save the Model

In [34]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'models/rand_forest_model.sav'
joblib.dump(rf, filename)

['models/rand_forest_model.sav']